In [ ]:
import os
import whisper
import torch
from pydub import AudioSegment
from pydub.silence import split_on_silence
from tqdm import tqdm
import ffmpeg
import subprocess

# Конфигурация
CONFIG = {
    "model_size": "medium",          # Выбор модели Whisper
    "min_chunk_duration": 30,        # Минимальная длина фрагмента (сек)
    "max_chunk_duration": 60,        # Максимальная длина фрагмента (сек)
    "silence_thresh": -40,           # Порог тишины в dB
    "min_silence_len": 800,          # Минимальная пауза (мс)
    "keep_silence": 300,             # Оставлять тишины по краям (мс)
    "output_folder": "video_chunks", # Папка для результатов
    "language": "ru"                 # Язык распознавания
}

class VideoProcessor:
    def __init__(self, config):
        self.config = config
        self.model = self._load_model()
        os.makedirs(config["output_folder"], exist_ok=True)

    def _load_model(self):
        device = "cuda" if torch.cuda.is_available() else "cpu"
        print(f"Загружаем модель Whisper ({self.config['model_size']}) на {device}...")
        return whisper.load_model(self.config["model_size"], device=device)

    def _extract_audio(self, video_path):
        audio_path = os.path.join(self.config["output_folder"], "temp_audio.wav")
        subprocess.run([
            "ffmpeg",
            "-i", video_path,
            "-ac", "1",
            "-ar", "16000",
            "-y",
            audio_path
        ], check=True)
        return audio_path

    def _process_audio(self, audio_path):
        """Обрабатывает аудио и возвращает фрагменты"""
        audio = AudioSegment.from_file(audio_path)
        audio = audio.normalize()

        print("Разделение аудио на фрагменты...")
        chunks = split_on_silence(
            audio,
            min_silence_len=self.config["min_silence_len"],
            silence_thresh=self.config["silence_thresh"],
            keep_silence=self.config["keep_silence"]
        )

        # Объединяем короткие фрагменты
        combined = []
        current = chunks[0]
        
        for chunk in tqdm(chunks[1:], desc="Объединение фрагментов"):
            duration = (len(current) + len(chunk)) / 1000
            if duration < self.config["max_chunk_duration"]:
                current += chunk
            else:
                if len(current) / 1000 >= self.config["min_chunk_duration"]:
                    combined.append(current)
                current = chunk

        if len(current) / 1000 >= self.config["min_chunk_duration"]:
            combined.append(current)

        return combined

    def _split_video(self, video_path, chunks):
        for i, chunk in enumerate(chunks):
            start = len(sum(chunks[:i], AudioSegment.empty())) / 1000
            end = start + len(chunk) / 1000
            output_path = os.path.join(self.config["output_folder"], f"chunk_{i:03d}.mp4")
            
            subprocess.run([
                "ffmpeg",
                "-i", video_path,
                "-ss", str(start),
                "-to", str(end),
                "-c", "copy",
                "-y",
                output_path
            ], check=True)

    def process(self, video_path):
        """Основной метод обработки"""
        try:
            # 1. Извлекаем аудио
            audio_path = self._extract_audio(video_path)
            
            # 2. Получаем фрагменты
            chunks = self._process_audio(audio_path)
            
            # 3. Нарезаем видео
            self._split_video(video_path, chunks)
            
            # 4. Транскрибация (опционально)
            if self.model:
                for i, chunk in enumerate(chunks):
                    chunk.export(f"temp.wav", format="wav")
                    result = self.model.transcribe(
                        "temp.wav",
                        language=self.config["language"],
                        word_timestamps=True
                    )
                    with open(os.path.join(self.config["output_folder"], f"chunk_{i:03d}.txt"), "w") as f:
                        f.write(result["text"])
                    os.remove("temp.wav")
            
            print(f"\nГотово! Сохранено {len(chunks)} фрагментов в {self.config['output_folder']}")
            
        finally:
            # Очистка временных файлов
            if os.path.exists(audio_path):
                os.remove(audio_path)

if __name__ == "__main__":
    processor = VideoProcessor(CONFIG)
    processor.process("../data/10minutes_lecture.mp4")  # Укажите ваш файл